In [1]:
#Import Dependencies
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import psycopg2
import time


In [2]:
#Import db password
from config import db_password

In [3]:
#file path
file_dir='C:/Users/hhngu/Desktop/BootCamp/Module_8/'

In [6]:
#Define files to load
def load_movie_file(wiki_file, kaggle_file, ratings):
    with open(f'{file_dir}/wikipedia.movies.json', mode='r') as file:
        wiki_movies_raw=json.load(file)
    kaggle_metadata=pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
    ratings=pd.read_csv(f'{file_dir}ratings.csv')
    
    #select movie that have director, imdb link and no. of episodes
    wiki_movies = [movie for movie in wiki_movies_raw
        if ('Director' in movie or 'Directed by' in movie) and 
               'imdb_link' in movie and 
               'No. of episodes' not in movie]
    wiki_movies_df=pd.DataFrame(wiki_movies)
    
    #Drop duplicate
    wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
    wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    #keep columns with <90% null
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
    